In [1]:
import numpy as np
import tensorflow as tf

from keras import Model
from keras.utils.data_utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout

from dataCleaning import load_and_preprocess_img

2023-04-27 08:33:07.164227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.12.0
Num GPUs Available:  1


2023-04-27 08:33:09.054556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 08:33:09.070143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 08:33:09.070305: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
def init_vgg16():
    vgg16 = VGG16(weights=None,
                  include_top=False,
                  input_shape=(224, 224, 3))

    # Create a new 'top' of the model of fully-connected layers for Places365
    top_model = vgg16.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu', name="fc1")(top_model)
    top_model = Dropout(0.5, name="drop_fc1")(top_model)
    top_model = Dense(4096, activation='relu', name="fc2")(top_model)
    top_model = Dropout(0.2, name="drop_fc2")(top_model)
    output_layer = Dense(365, activation='softmax', name="predictions")(top_model)

    model = Model(inputs=vgg16.input,
                  outputs=output_layer,
                  name="vgg16-places365")

    WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'

    weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5',
                        WEIGHTS_PATH,
                        cache_subdir='models')

    model.load_weights(weights_path)

    return model

In [10]:
model = init_vgg16()
model.summary()

Model: "vgg16-places365"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)   

In [25]:
img = load_and_preprocess_img('test_img.jpg')
img = np.expand_dims(img, 0)
img.shape

(1, 224, 224, 3)

In [26]:
preds = model.predict(img)[0]

1/1 [==============================] - 0s 91ms/step


In [28]:
no_of_predictions = 5
top_preds = np.argsort(preds)[::-1][0:no_of_predictions]

In [29]:
import os

# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

--2023-04-20 13:58:05--  https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6833 (6.7K) [text/plain]
Saving to: ‘categories_places365.txt’

     0K ......                                                100% 19.3M=0s

2023-04-20 13:58:05 (19.3 MB/s) - ‘categories_places365.txt’ saved [6833/6833]



In [32]:
type(top_preds)

numpy.ndarray

In [31]:
print('--PREDICTED SCENE CATEGORIES:')
# output the prediction
for i in range(0, no_of_predictions):
    print(classes[top_preds[i]])

--PREDICTED SCENE CATEGORIES:
park
forest_path
botanical_garden
orchard
forest/broadleaf


In [33]:
def get_prediction_labels(predictions: np.ndarray) -> list:
    l = []
    for i in range(len(predictions)):
        l.append(classes[predictions[i]])

    return l

In [34]:
get_prediction_labels(top_preds)

['park', 'forest_path', 'botanical_garden', 'orchard', 'forest/broadleaf']